In [83]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import missingno as msno
%matplotlib inline

In [84]:
df=pd.read_csv("https://raw.githubusercontent.com/kunan-sa/mentoria_2024/master/dataset/base_asistencia_anon.csv", encoding='utf8')
df=df.drop(columns=['Unnamed: 0'])
df_base = df.copy()

# RENOMBRAR COLUMNAS

In [85]:
df_base=df_base.rename(columns={'action_A_count':'qt_sac_pac_ac',"action_B_count":"qt_canc_pac_ac",'action_C_count':'qt_cons_pac'})
df_base.head(2)

,doc_id,doc_full_name,msp_name,msp_id,hos_id,heq_id,hin_name,hin_id,age_avg,dni_asistance_rate,...,msp_asistance_rate,hosheq_asistance_rate,app_start_dt,qt_sac_pac_ac,qt_canc_pac_ac,qt_cons_pac,app_days_gap,patient_id,event_id,attendance
0,215,Carmen Sánchez,Neurologa,159,Hospital Dragonstone,3,PAMI INSTNACSERVSOCJUBY PENSPAMI,0,77.0,72.727273,...,71.085359,69.875045,2023-06-23 15:40:00.000,11,2,4,39,75571,290588,0
1,1855,Sofía García,CARDIOLOGIA INFANTIL,141,Instituto Harrenhal,12,OSDE,0,7.0,50.000000,...,75.919732,85.898672,2023-07-18 09:00:00.000,1,0,0,48,374820,3590242,0


# VALORES FALTANTES --> MEJORAR LAS VARS PARA FALTANTES


In [86]:
# Control
df_faltantes=df_base.isnull().sum()
df_faltantes

,0
doc_id,0
doc_full_name,0
msp_name,0
msp_id,0
hos_id,0
heq_id,0
hin_name,0
hin_id,0
age_avg,0
dni_asistance_rate,54


In [87]:
# Promedio de la Tasa de Asistencia por Hospital
mean_hosp = df_base.copy()[['hos_id','dni_asistance_rate']]
mean_hosp = mean_hosp[~mean_hosp['dni_asistance_rate'].isnull()]
mean_hosp = mean_hosp.groupby(['hos_id',],as_index=False)['dni_asistance_rate'].mean()
mean_hosp = mean_hosp.rename(columns={'dni_asistance_rate':'mean_hos_asis'})
mean_hosp

,hos_id,mean_hos_asis
0,Centro Winterfell,72.434063
1,Centro médico Casterly Rock,81.771809
2,Consultorios Poniente,8.879695
3,Dorne,77.752431
4,Hospital Dragonstone,68.508804
5,Instituto Harrenhal,75.625496


In [88]:
#Promedio de la Tasa de Asistencia por Médico y Sede
mean_doc=df_base.copy()[['hos_id',"heq_id",'doc_asistance_rate']]
mean_doc=mean_doc[~mean_doc['doc_asistance_rate'].isnull()]
mean_doc=mean_doc.groupby(['hos_id','heq_id'],as_index=False)['doc_asistance_rate'].mean()
mean_doc=mean_doc.rename(columns={'doc_asistance_rate':'mean_doc_asis'})
mean_doc

,hos_id,heq_id,mean_doc_asis
0,Centro Winterfell,2,79.745401
1,Centro Winterfell,3,72.386595
2,Centro Winterfell,4,78.190743
3,Centro médico Casterly Rock,2,85.245354
4,Centro médico Casterly Rock,5,81.263634
5,Consultorios Poniente,1,19.470896
6,Dorne,3,77.369469
7,Hospital Dragonstone,3,69.427539
8,Instituto Harrenhal,4,78.749156
9,Instituto Harrenhal,6,79.859384


In [89]:
# Promedio de la Tasa de Asistencia por Especialidad en Hospital y Sede
mean_esp=df_base.copy()[['hos_id',"heq_id",'msp_asistance_rate']] #msp_name
mean_esp=mean_esp[~mean_esp['msp_asistance_rate'].isnull()]
mean_esp=mean_esp.groupby(['hos_id','heq_id'],as_index=False)['msp_asistance_rate'].mean()
mean_esp=mean_esp.rename(columns={'msp_asistance_rate':'mean_esp_asis'})
mean_esp

,hos_id,heq_id,mean_esp_asis
0,Centro Winterfell,2,78.888378
1,Centro Winterfell,3,76.797696
2,Centro Winterfell,4,78.841750
3,Centro médico Casterly Rock,2,84.017314
4,Centro médico Casterly Rock,5,84.432064
5,Consultorios Poniente,1,20.692403
6,Dorne,3,77.332868
7,Hospital Dragonstone,3,69.570868
8,Instituto Harrenhal,4,80.873776
9,Instituto Harrenhal,6,81.469948


In [90]:
# Completamos los valores nulos usando los promedios ya calculados arriba
df_mean=df_base.copy()
df_mean=df_mean.merge(mean_hosp,how='left',on='hos_id')
df_mean=df_mean.merge(mean_doc,how='left',on=['hos_id','heq_id'])
df_mean=df_mean.merge(mean_esp,how='left',on=['hos_id','heq_id'])

df_mean['dni_asistance_rate']=np.where(df_mean['dni_asistance_rate'].isnull(),df_mean['mean_hos_asis'],df_mean['dni_asistance_rate'])
df_mean['doc_asistance_rate']=np.where(df_mean['doc_asistance_rate'].isnull(),df_mean['mean_doc_asis'],df_mean['doc_asistance_rate'])
df_mean['msp_asistance_rate']=np.where(df_mean['msp_asistance_rate'].isnull(),df_mean['mean_esp_asis'],df_mean['msp_asistance_rate'])
df_base=df_mean.copy()
df_base.head(2)

,doc_id,doc_full_name,msp_name,msp_id,hos_id,heq_id,hin_name,hin_id,age_avg,dni_asistance_rate,...,qt_sac_pac_ac,qt_canc_pac_ac,qt_cons_pac,app_days_gap,patient_id,event_id,attendance,mean_hos_asis,mean_doc_asis,mean_esp_asis
0,215,Carmen Sánchez,Neurologa,159,Hospital Dragonstone,3,PAMI INSTNACSERVSOCJUBY PENSPAMI,0,77.0,72.727273,...,11,2,4,39,75571,290588,0,68.508804,69.427539,69.570868
1,1855,Sofía García,CARDIOLOGIA INFANTIL,141,Instituto Harrenhal,12,OSDE,0,7.0,50.000000,...,1,0,0,48,374820,3590242,0,75.625496,86.026745,86.187105


In [91]:
df_faltantes=df_base.isnull().sum()
df_faltantes

,0
doc_id,0
doc_full_name,0
msp_name,0
msp_id,0
hos_id,0
heq_id,0
hin_name,0
hin_id,0
age_avg,0
dni_asistance_rate,0


# CAMBIOS EN ESPECIALIDADES

In [92]:
#1. IMPORTANTES
df_base['msp_name'] = df_base['msp_name'].str.upper()
df_base['msp_name'] = df_base['msp_name'].str.replace(r'LOGA$', 'LOGIA', regex=True)
df_base['msp_name'] = df_base['msp_name'].str.replace('CIRUGA', 'CIRUGIA')

In [93]:
# CAMBIOS GENERALES aplicados al DF df_base
df_base['msp_name'] = df_base['msp_name'].str.replace('ERGOMETRAS', 'ERGOMETRIA')
df_base['msp_name'] = df_base['msp_name'].str.replace('NUTRICIN', 'NUTRICION', regex=False)
df_base['msp_name'] = df_base['msp_name'].str.replace('NEUROCIRUGA', 'NEUROCIRUGIA', regex=False)
df_base['msp_name'] = df_base['msp_name'].str.replace('GINECOLGICA', 'GINECOLOGICA')
df_base['msp_name'] = df_base['msp_name'].str.replace('DIABETES', 'DIABETOLOGIA')
df_base.loc[df_base['msp_name'] == 'NEUROLOGA PEDIATRICA', 'msp_name'] = 'NEUROLOGIA'
df_base['msp_name'] = df_base['msp_name'].str.replace('PLANTILLAS', 'ORTOPEDIA')
df_base['msp_name'] = df_base['msp_name'].str.replace('MEDICINA CLINICA', 'CLINICA MEDICA')
df_base['msp_name'] = df_base['msp_name'].str.replace('CONTROL ANUAL MUJER', 'GINECOLOGIA')
df_base['msp_name'] = df_base['msp_name'].str.replace('ORL', 'OTORRINOLARINGOLOGIA')
df_base['msp_name'] = df_base['msp_name'].str.replace('FLEBOLOGA CONS EXT', 'FLEBOLOGIA')
df_base['msp_name'] = df_base['msp_name'].str.replace('OSTEOPATA', 'OSTEOPATIA')

# CIRUGIAS
df_base.loc[df_base['msp_name'] == 'CIRUGIA', 'msp_name'] = 'CIRUGIA GENERAL'
df_base['msp_name'] = df_base['msp_name'].str.replace('CIRUGIA PLASTICA Y REPARADORA', 'CIRUGIA PLASTICA')
df_base.loc[df_base['msp_name'] == 'CIRUGIA TORAX', 'msp_name'] = 'CIRUGIA TORAXICA'
df_base.loc[df_base['msp_name'] == 'CABEZA Y CUELLO', 'msp_name'] = 'CIRUGIA DE CABEZA Y CUELLO'
df_base.loc[df_base['msp_name'] == 'CIRUGIA PLSTICA Y QUEMADOS', 'msp_name'] = 'CIRUGIA PLASTICA'

In [94]:
#SUB CAMBIOS segun criterios tomados
# 3. SUB CAMBIOS

df_base['msp_name'] = df_base['msp_name'].str.replace('VASCULAR PERIFRICO', 'VASCULAR')

#Las Oea (otoemisiones Acusticas) la realizan los Fonoaudiólogos pero con especialidad en audición!--
#Criterio: se separara FONOAUDIOLOGIA de FONOAUDIOLOGIA ESPECIALIDAD AUDICION
df_base['msp_name'] = df_base['msp_name'].str.replace('FONOAUDIOLOGIA OTOEMISIONES ACUSTICAS OEA', 'FONOAUDIOLOGIA ESPECIALIDAD AUDICION')
df_base['msp_name'] = df_base['msp_name'].str.replace('FONOAUD OTOEMISIONES ACUSTICAS OEA', 'FONOAUDIOLOGIA ESPECIALIDAD AUDICION')
df_base['msp_name'] = df_base['msp_name'].str.replace('OTOEMISIONES ACUSTICAS OEA', 'FONOAUDIOLOGIA ESPECIALIDAD AUDICION')

# La mamografía es un tipo de imagen médica especializada-Los mastólogos son médicos dedicados al tratamiento de las patologías mamarias
df_base['msp_name'] = df_base['msp_name'].str.replace('MAMOGRAFIA', 'MASTOLOGIA')

# MAPA, Holter, Ecocardiograma y Ergometría son pruebas y procedimientos diagnósticos
# relacionados con el corazón y el sistema cardiovascular. Estos términos pueden ser unificados bajo la especialidad de Cardiología.
# 'MAPA'y 'HOLTER','ECOCARDIOGRAMA' y 'ERGOMETRIA'---> = 'CARDIOLOGIA'
df_base['msp_name'] = df_base['msp_name'].str.replace('HOLTER', 'CARDIOLOGIA')
df_base['msp_name'] = df_base['msp_name'].str.replace('MAPA', 'CARDIOLOGIA')
df_base['msp_name'] = df_base['msp_name'].str.replace('ECOCARDIOGRAMA', 'CARDIOLOGIA')
df_base['msp_name'] = df_base['msp_name'].str.replace('CARDIOLOGIA ERGOMETRIA', 'CARDIOLOGIA')
df_base.loc[df_base['msp_name'] == 'ERGOMETRIA', 'msp_name'] = 'CARDIOLOGIA'

# ESPIROMETRAS = ESPIROMETRIA --La espirometría es una prueba que utilizan los médicos para medir qué tan bien están funcionando los pulmones
# los especialistas en Neumonología y Cirugía Torácica, son los encargados de realizar este estudio
# Criterio: Correcion del nombre del estudio 2)Encasillar el estudio en la especialidad 'NEUMONOLOGIA'
df_base['msp_name'] = df_base['msp_name'].str.replace('ESPIROMETRAS', 'NEUMONOLOGIA')


# OUTLIERS

In [95]:
df_base = df_base.copy()
cols_to_analyze = ['hosheq_asistance_rate', 'qt_sac_pac_ac', 'qt_canc_pac_ac', 'qt_cons_pac']

percentiles = df_base[cols_to_analyze].quantile(0.95)

for col in cols_to_analyze:
    percentile_95_value = percentiles[col]
    df_base[col] = np.where(df_base[col] > percentile_95_value, percentile_95_value, df_base[col])

# Filtrar la columna 'age_avg'
df_base = df_base[df_base['age_avg'] != 2024]

# Filtrar la columna 'age_avg'
df_base = df_base[df_base['hos_id'] != 'Consultorios Poniente']

df_base.head(3)


,doc_id,doc_full_name,msp_name,msp_id,hos_id,heq_id,hin_name,hin_id,age_avg,dni_asistance_rate,...,qt_sac_pac_ac,qt_canc_pac_ac,qt_cons_pac,app_days_gap,patient_id,event_id,attendance,mean_hos_asis,mean_doc_asis,mean_esp_asis
0,215,Carmen Sánchez,NEUROLOGIA,159,Hospital Dragonstone,3,PAMI INSTNACSERVSOCJUBY PENSPAMI,0,77.0,72.727273,...,11.0,2.0,4.0,39,75571,290588,0,68.508804,69.427539,69.570868
1,1855,Sofía García,CARDIOLOGIA INFANTIL,141,Instituto Harrenhal,12,OSDE,0,7.0,50.000000,...,1.0,0.0,0.0,48,374820,3590242,0,75.625496,86.026745,86.187105
2,215,Carmen Sánchez,NEUROLOGIA,159,Hospital Dragonstone,3,PAMI INSTNACSERVSOCJUBY PENSPAMI,0,60.0,37.500000,...,10.0,0.0,2.0,33,72497,320006,0,68.508804,69.427539,69.570868


In [96]:
df_check=df_base.copy()['hos_id'].unique()
df_check

array(['Hospital Dragonstone', 'Instituto Harrenhal',
       'Centro médico Casterly Rock', 'Centro Winterfell', 'Dorne'],
      dtype=object)

In [97]:
# df_base=df_base.copy()
# df_base=df_base[df_base['age_avg']!=2024]
# df_base=df_base[df_base['hosheq_asistance_rate']>=25]
# df_base=df_base[df_base['qt_sac_pac_ac']<=70]
# df_base=df_base[df_base['qt_canc_pac_ac']<=60]
# df_base=df_base[df_base['qt_cons_pac']<=20]
# df_base.head(3)

In [98]:
cols_sel=['age_avg', 'dni_asistance_rate', 'doc_asistance_rate','msp_asistance_rate', 'hosheq_asistance_rate','qt_sac_pac_ac', 'qt_canc_pac_ac', 'qt_cons_pac', 'app_days_gap','attendance']

In [99]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df_box = df_base[cols_sel]
fig = make_subplots(rows=1, cols=len(cols_sel), subplot_titles=cols_sel)

for i, column in enumerate(cols_sel):
    box = go.Box(y=df_box[column], name=column)
    fig.add_trace(box, row=1, col=i+1)

fig.update_layout(height=800, width=2000)
fig.show()

# COLS ADICIONALES

In [100]:
df_base['app_start_dt'] = pd.to_datetime(df_base['app_start_dt'])
df_base['hora'] = df_base['app_start_dt'].dt.hour
df_base['turno']=df_base['hora'].apply(lambda x: 'mañana' if 6 <= x < 12 else 'tarde' if 12 <= x < 18 else 'noche')
df_base.head(2)

,doc_id,doc_full_name,msp_name,msp_id,hos_id,heq_id,hin_name,hin_id,age_avg,dni_asistance_rate,...,qt_cons_pac,app_days_gap,patient_id,event_id,attendance,mean_hos_asis,mean_doc_asis,mean_esp_asis,hora,turno
0,215,Carmen Sánchez,NEUROLOGIA,159,Hospital Dragonstone,3,PAMI INSTNACSERVSOCJUBY PENSPAMI,0,77.0,72.727273,...,4.0,39,75571,290588,0,68.508804,69.427539,69.570868,15,tarde
1,1855,Sofía García,CARDIOLOGIA INFANTIL,141,Instituto Harrenhal,12,OSDE,0,7.0,50.000000,...,0.0,48,374820,3590242,0,75.625496,86.026745,86.187105,9,mañana


In [101]:
# Cantidad total de pacientes
cant_pacientes = len(df_base['patient_id'].unique())

# Determinar si cada patient_id es un duplicado
lista_turnos_con_duplicidad = df_base['patient_id'].duplicated(keep='first')

# Filtrar los 'patient_id' que son duplicados
turnos_duplicados = df_base.loc[lista_turnos_con_duplicidad, 'patient_id']

pacientes_con_mas_de_un_turno = turnos_duplicados.unique()

if turnos_duplicados.empty:
    print("Todos los pacientes sacaron turno por primera vez.")
else:
    print("Los siguientes pacientes han sacado turnos más de una vez:")
    print(pacientes_con_mas_de_un_turno)

print("Total de pacientes:",cant_pacientes)


cant_pacientes_nuevos = cant_pacientes - len(pacientes_con_mas_de_un_turno)

print("Total de pacientes que sacaron turno por primera vez:",cant_pacientes_nuevos)

print("Total de pacientes que han sacado turno en otra oportunidad:", len(pacientes_con_mas_de_un_turno))

Los siguientes pacientes han sacado turnos más de una vez:
[657289  72755  95412 ...  10535  40856 101377]
Total de pacientes: 27737
Total de pacientes que sacaron turno por primera vez: 18911
Total de pacientes que han sacado turno en otra oportunidad: 8826


In [102]:
# Agregar la nueva columna 'paciente_nuevo' basada en si es duplicado o no
df_base['paciente_nuevo'] = lista_turnos_con_duplicidad.map({True: 'no', False: 'si'})

In [103]:
df_base.head()

,doc_id,doc_full_name,msp_name,msp_id,hos_id,heq_id,hin_name,hin_id,age_avg,dni_asistance_rate,...,app_days_gap,patient_id,event_id,attendance,mean_hos_asis,mean_doc_asis,mean_esp_asis,hora,turno,paciente_nuevo
0,215,Carmen Sánchez,NEUROLOGIA,159,Hospital Dragonstone,3,PAMI INSTNACSERVSOCJUBY PENSPAMI,0,77.0,72.727273,...,39,75571,290588,0,68.508804,69.427539,69.570868,15,tarde,si
1,1855,Sofía García,CARDIOLOGIA INFANTIL,141,Instituto Harrenhal,12,OSDE,0,7.0,50.000000,...,48,374820,3590242,0,75.625496,86.026745,86.187105,9,mañana,si
2,215,Carmen Sánchez,NEUROLOGIA,159,Hospital Dragonstone,3,PAMI INSTNACSERVSOCJUBY PENSPAMI,0,60.0,37.500000,...,33,72497,320006,0,68.508804,69.427539,69.570868,13,tarde,si
3,3,Arizona Robbins,GINECOLOGIA Y OBSTETRICIA,14,Centro médico Casterly Rock,5,OSDE,0,34.0,50.000000,...,1,12341,597903,1,81.771809,81.263634,84.432064,18,noche,si
4,43,Graham Pritchard,GINECOLOGIA,151,Hospital Dragonstone,3,PAMI INSTNACSERVSOCJUBY PENSPAMI,0,60.0,100.000000,...,13,86202,305566,1,68.508804,69.427539,69.570868,16,tarde,si


In [104]:
from google.colab import files

nombre_archivo = 'base_turnos_hospitales.csv'
df_base.to_csv(nombre_archivo, index=False)

files.download(nombre_archivo)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>